<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/miscellaneous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === System + training configuration snapshot for "Table 3" ===
import os, re, csv, json, subprocess, shutil, pandas as pd
from pathlib import Path

# ---- From your code (edit here only if you changed them) ----
PLATFORM = "Google Colab"
BATCH_SIZE = 32
EPOCHS_CONFIGURED = 40
OPTIMIZER = "Adamax"
LEARNING_RATE = 0.0005
LOSS_FUNCTION = "categorical_crossentropy"   # because you use 2 classes with class_mode='categorical'
INCLUDE_TOP = False
TRAINABLE_BACKBONE = True
PRETRAINED_WEIGHTS = "ImageNet"

# Paths where your logs/models were saved (from your code)
BASE   = "/content/drive/MyDrive/DiabeticProject"
SAVE_DIR = f"{BASE}/saved_models"

# ---- System info helpers ----
def shell(cmd):
    return subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.STDOUT).strip()

def get_disk_free_gb():
    out = shell("df -k / | tail -1")
    parts = out.split()
    # parts: Filesystem,1K-blocks,Used,Available,Use%,Mounted
    avail_kb = int(parts[3])
    return round(avail_kb / (1024**2), 2)  # GB

def get_ram_gb():
    # MemTotal from /proc/meminfo (kB)
    with open("/proc/meminfo") as f:
        for line in f:
            if line.startswith("MemTotal:"):
                kb = int(re.findall(r"\d+", line)[0])
                return round(kb / (1024**2), 2)  # GB
    return None

def get_gpu_and_cuda():
    try:
        smi = shell("nvidia-smi --query-gpu=name,driver_version,memory.total --format=csv,noheader")
        name, driver, mem = [s.strip() for s in smi.split(",")]
        # CUDA version (runtime)
        cudaver = None
        try:
            cudaver = shell("nvcc --version | tail -1")
            cudaver = re.search(r"release\s+([0-9.]+)", cudaver).group(1)
        except Exception:
            # fallback from nvidia-smi header
            head = shell("nvidia-smi | head -n1")
            m = re.search(r"CUDA Version:\s*([0-9.]+)", head)
            cudaver = m.group(1) if m else "Unknown"
        return name, cudaver, mem
    except Exception:
        return "None/CPU", "N/A", "N/A"

GPU_NAME, CUDA_VERSION, GPU_MEM = get_gpu_and_cuda()

# ---- Count actual epochs from your per-model training logs, if present ----
def count_epochs_from_csv(log_csv):
    if not os.path.exists(log_csv):
        return None
    # Count non-header rows (unique epoch lines)
    with open(log_csv, "r", encoding="utf-8") as f:
        r = csv.DictReader(f)
        epochs = 0
        for _ in r:
            epochs += 1
    return epochs or None

# Try to summarize epochs for the models you trained
epoch_details = []
if os.path.isdir(SAVE_DIR):
    for root, dirs, files in os.walk(SAVE_DIR):
        if "training_log.csv" in files:
            ep = count_epochs_from_csv(os.path.join(root, "training_log.csv"))
            model_id = Path(root).name
            epoch_details.append((model_id, ep))

# Build the Table 3 dataframe
rows = [
    (1, "Platform", PLATFORM),
    (2, "Disk space (free)", f"{get_disk_free_gb()} GB"),
    (3, "RAM", f"{get_ram_gb()} GB"),
    (4, "GPU", f"{GPU_NAME} ({GPU_MEM} total)"),
    (5, "CUDA version", CUDA_VERSION),
    (6, "Batch Size", BATCH_SIZE),
    (7, "Epochs used (configured)", EPOCHS_CONFIGURED),
    (8, "Optimizer", OPTIMIZER),
    (9, "Learning Rate", LEARNING_RATE),
    (10, "Loss function", LOSS_FUNCTION),
    (11, "Include_top (Backbones)", str(INCLUDE_TOP)),
    (12, "Trainable (Transfer Learning)", str(TRAINABLE_BACKBONE)),
    (13, "Pre-trained Weights", PRETRAINED_WEIGHTS),
]

# If we found actual trained epochs in logs, append a rollup
if epoch_details:
    # e.g., "EfficientNetV2B0_single: 40, DenseNet121_single: 38"
    pretty = ", ".join([f"{mid}: {ep if ep is not None else '—'}" for mid, ep in epoch_details])
    rows.append((14, "Training Epochs (actual from logs)", pretty))
else:
    rows.append((14, "Training Epochs (actual from logs)", "Logs not found"))

df_table3 = pd.DataFrame(rows, columns=["S.No", "Hardware/Experimental Parameter", "Configuration/Value"])
df_table3


,S.No,Hardware/Experimental Parameter,Configuration/Value
0,1,Platform,Google Colab
1,2,Disk space (free),73.57 GB
2,3,RAM,12.67 GB
3,4,GPU,Tesla T4 (15360 MiB total)
4,5,CUDA version,Unknown
5,6,Batch Size,32
6,7,Epochs used (configured),40
7,8,Optimizer,Adamax
8,9,Learning Rate,0.0005
9,10,Loss function,categorical_crossentropy


In [ ]:
# === Robust CUDA detection for Colab ===
import os, re, subprocess, json

def sh(cmd):
    try:
        return subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.STDOUT).strip()
    except Exception:
        return None

def cuda_versions():
    out = {
        "gpu_name": None,
        "driver_cuda_header": None,   # from nvidia-smi header ("CUDA Version:")
        "nvcc_version": None,         # toolkit version if nvcc installed
        "cuda_dir_version": None,     # from /usr/local/cuda/version.txt
        "torch_cuda": None,           # CUDA that torch was compiled against
        "tf_cuda": None,              # CUDA that TF was built against
        "cudnn_from_tf": None,        # cuDNN from TF build info (if available)
    }

    # GPU name + nvidia-smi header CUDA
    smi = sh("nvidia-smi")
    if smi:
        m = re.search(r"CUDA Version:\s*([0-9.]+)", smi)
        out["driver_cuda_header"] = m.group(1) if m else None
        m2 = re.search(r"^\| NVIDIA-SMI.*?\|$", smi, re.MULTILINE)
        out["gpu_name"] = sh("nvidia-smi --query-gpu=name --format=csv,noheader")

    # nvcc (toolkit)
    nvcc = sh("nvcc --version")
    if nvcc:
        m = re.search(r"release\s+([0-9.]+)", nvcc)
        out["nvcc_version"] = m.group(1) if m else None

    # /usr/local/cuda/version.txt
    if os.path.exists("/usr/local/cuda/version.txt"):
        try:
            with open("/usr/local/cuda/version.txt") as f:
                txt = f.read()
            m = re.search(r"CUDA Version\s*([0-9.]+)", txt)
            out["cuda_dir_version"] = m.group(1) if m else txt.strip()
        except Exception:
            pass

    # PyTorch build CUDA
    try:
        import torch
        out["torch_cuda"] = torch.version.cuda
    except Exception:
        pass

    # TensorFlow build CUDA + cuDNN
    try:
        import tensorflow as tf
        bi = tf.sysconfig.get_build_info()
        # New TF returns dict with 'cuda_version' and 'cudnn_version' sometimes
        out["tf_cuda"] = bi.get("cuda_version") or bi.get("cuda_version_number")
        out["cudnn_from_tf"] = bi.get("cudnn_version") or bi.get("cudnn_version_number")
    except Exception:
        pass

    return out

info = cuda_versions()
print(json.dumps(info, indent=2))


{
  "gpu_name": "Tesla T4",
  "driver_cuda_header": "12.4",
  "nvcc_version": "12.5",
  "cuda_dir_version": null,
  "torch_cuda": "12.6",
  "tf_cuda": "12.5.1",
  "cudnn_from_tf": "9"
}
